# Calculating a team's running average for each metric

It is inaccurate to say that all the averaged data cam be applied to yearly team matchups when for example when simulating betting in 2008 you as a better cannot know the future performace of a 2010 team. For this reason, it is important to consider runnign averages to make a more realistic and accutate model

This notebook works in developing running averaged metrics per teams

We would want to calculate the stats for both team A and team B in every game up to but not including their match against one another to make running average informed bets

In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt

## Data Imports

In [2]:
teams_df = pd.read_csv("../data//Kaggle-Data/MDataFiles_Stage1/MTeams.csv")
teams_df = teams_df.drop(["FirstD1Season","LastD1Season"], axis=1)
teams_df.head()

,TeamID,TeamName
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M


In [3]:
detailed_season_df = pd.read_csv("../data//Kaggle-Data/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv")
detailed_season_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [4]:
team_stats_batch1 = detailed_season_df[['Season', 'DayNum', 'WTeamID', 'WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']].copy()
team_stats_batch1.columns = ['Season', 'DayNum', 'TeamID', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
team_stats_batch1.head()

,Season,DayNum,TeamID,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,2003,10,1104,68,27,58,3,14,11,18,14,24,13,23,7,1,22
1,2003,10,1272,70,26,62,8,20,10,19,15,28,16,13,4,4,18
2,2003,11,1266,73,24,58,8,18,17,29,17,26,15,10,5,2,25
3,2003,11,1296,56,18,38,3,9,17,31,6,19,11,12,14,2,18
4,2003,11,1400,77,30,61,6,14,11,13,17,22,12,14,4,4,20


In [5]:
team_stats_batch2 = detailed_season_df[['Season', 'DayNum', 'LTeamID', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']].copy()
team_stats_batch2.columns = ['Season', 'DayNum', 'TeamID', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF'] 
team_stats_batch2.head()

,Season,DayNum,TeamID,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,2003,10,1328,62,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1393,63,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1437,61,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1457,50,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1208,71,24,62,6,16,17,27,21,15,12,10,7,1,14


In [6]:
#sanity check
a = team_stats_batch1.shape[0]
b = team_stats_batch2.shape[0]
print(a, b, a+b)

100423 100423 200846


In [7]:
team_stats = pd.merge(team_stats_batch1, team_stats_batch2, how='outer')
team_stats.head()

,Season,DayNum,TeamID,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,2003,10,1104,68,27,58,3,14,11,18,14,24,13,23,7,1,22
1,2003,10,1272,70,26,62,8,20,10,19,15,28,16,13,4,4,18
2,2003,11,1266,73,24,58,8,18,17,29,17,26,15,10,5,2,25
3,2003,11,1296,56,18,38,3,9,17,31,6,19,11,12,14,2,18
4,2003,11,1400,77,30,61,6,14,11,13,17,22,12,14,4,4,20


In [8]:
#sanity check
team_stats.shape[0]

200846

# How to use the running_avg_stats function

## Parameters

1. df = upload 'DataMining-FinalProject/data/CleanedTeamResults.csv' into a pandas dataframe
2. season = the Season of the current matchup the model is predicting as a integer value
3. daynum = the DayNum of the current matchup the model is predicting as an integer value
4. teamA_ID = the TeamID of the first team in the current matchup the model is predicting as a integer value
5. teamB_ID = the TeamID of the second team in the current matchup the model is predicting as a integer value

## Output

1. teamA = a pandas.descibe() data frame that is returned with mean/average summary stats of the first team's performance up to (but not including) the specific season and daynum parameter
2. teamB = a pandas.descibe() data frame that is returned with mean/average summary stats of the second team's performance up to (but not including) the specific season and daynum parameter

- Score:        AVERAGE GAME SCORE FOR THIS TEAM UP TO THE CURRENT GAME
- FGM:          AVERAGE FIELD GOAL MADE FOR THIS TEAM UP TO THE CURRENT GAME
- FGA:          AVERAGE FIELD GOAL ATTEMPTED FOR THIS TEAM UP TO THE CURRENT GAME
- FGM3:         AVERAGE 3-POINTER MADE FOR THIS TEAM UP TO THE CURRENT GAME
- FGA3:         AVERAGE 3-POINTER ATTEMPTED FOR THIS TEAM UP TO THE CURRENT GAME
- FTM:          AVERAGE FREE THROWS MADE FOR THIS TEAM UP TO THE CURRENT GAME
- FTA:          AVERAGE FREE THROWS ATTEMPTED FOR THIS TEAM UP TO THE CURRENT GAME
- OR:           AVERAGE OFFENSIVE REBOUNDS FOR THIS TEAM UP TO THE CURRENT GAME
- DR:           AVERAGE DEFENSIVE REBOUNDS FOR THIS TEAM UP TO THE CURRENT GAME
- Ast:          AVERAGE ASSISTS FOR THIS TEAM UP TO THE CURRENT GAME
- TO:           AVERAGE TURNOVERS FOR THIS TEAM UP TO THE CURRENT GAME
- Stl:          AVERAGE STEALS FOR THIS TEAM UP TO THE CURRENT GAME
- Blk:          AVERAGE BLOCKS FOR THIS TEAM UP TO THE CURRENT GAME
- PF:           AVERAGE PERSONAL FOULS COMMITTED MADE FOR THIS TEAM UP TO THE CURRENT GAME
- EFGP:         AVERAGE EFFECTIVE FIELD GOAL PERCENTAGE FOR THIS TEAM UP TO THE CURRENT GAME
- TP:           AVERAGE TURNOVER PERCENTAGE FOR THIS TEAM UP TO THE CURRENT GAME
- ORP:          AVERAGE OFFENSIVE REBOUND PERCENTAGE FOR THIS TEAM UP TO THE CURRENT GAME (SPECIFIC TO THE MATCHUP)
- FTR:          AVERAGE FREE THROW RATE FOR THIS TEAM UP TO THE CURRENT GAME
- 4Factor:      AVERAGE FOUR FACTOR SCORE FOR THIS TEAM UP TO THE CURRENT GAME (SPECIFIC TO THE MATCHUP)

In [9]:
def running_avg_stats(df, season, daynum, teamA_ID, teamB_ID):
    running_data = df[df['Season'] <= season]
    
    teamA = running_data[running_data['TeamID'] == teamA_ID]
    teamA = teamA[teamA['DayNum'] < daynum]
    teamA = teamA.describe().loc['mean']
    
    teamB = running_data[running_data['TeamID'] == teamB_ID]
    teamB = teamB[teamB['DayNum'] < daynum]
    teamB = teamB.describe().loc['mean']
    
    # Adding in the current matchup four factor scores
    
    # Effective Field Goal Percentage
    # FGM + (0.5 * FGM3) / FGA
    teamA['EFGP'] = (teamA['FGM'] + (0.5 * teamA['FGM3'])) / teamA['FGA']
    teamB['EFGP'] = (teamB['FGM'] + (0.5 * teamB['FGM3'])) / teamB['FGA']
    
    #Turnover Percentage
    #TO / (FGA + (0.44 * FTA) + TO)
    teamA['TP'] = teamA['TO'] / (teamA['FGA'] + (0.44 * teamA['FTA']) + teamA['TO'])
    teamB['TP'] = teamB['TO'] / (teamB['FGA'] + (0.44 * teamB['FTA']) + teamB['TO'])
    
    # Offensive Rebound Percentage
    # OR / OR + OpposingTeam_DR
    teamA['ORP'] = teamA['OR'] / (teamA['OR']  + teamB['DR'])
    teamB['ORP'] = teamB['OR'] / (teamB['OR']  + teamA['DR'])
    
    # Free Throw Rate
    # FTM / FGA
    teamA['FTR'] = teamA['FTM'] / teamA['FGA']
    teamB['FTR'] = teamB['FTM'] / teamB['FGA']
    
    #Four Factor Score
    teamA['4Factor'] = (0.4 * teamA['EFGP']) - (0.25 * teamA['TP']) + (0.2 * teamA['ORP']) + (0.15 * teamA['FTR'])
    teamB['4Factor'] = (0.4 * teamB['EFGP']) - (0.25 * teamB['TP']) + (0.2 * teamB['ORP']) + (0.15 * teamB['FTR'])
    
    return teamA, teamB

In [10]:
A, B = running_avg_stats(team_stats, 2004, 15, 1104, 1272)

In [11]:
print(A)

Season     2003.500000
DayNum       10.500000
TeamID     1104.000000
Score        65.000000
FGM          24.500000
FGA          57.500000
FGM3          3.000000
FGA3         14.000000
FTM          13.000000
FTA          20.500000
OR           14.500000
DR           23.500000
Ast          10.500000
TO           17.500000
Stl           5.500000
Blk           2.500000
PF           24.500000
EFGP          0.452174
TP            0.208284
ORP           0.333333
FTR           0.226087
4Factor       0.229378
Name: mean, dtype: float64


In [12]:
print(B)

Season     2003.500000
DayNum       10.000000
TeamID     1272.000000
Score        73.000000
FGM          26.500000
FGA          65.000000
FGM3          9.000000
FGA3         24.500000
FTM          11.000000
FTA          24.000000
OR           17.000000
DR           29.000000
Ast          13.500000
TO           14.500000
Stl           3.500000
Blk           4.500000
PF           22.500000
EFGP          0.476923
TP            0.161004
ORP           0.419753
FTR           0.169231
4Factor       0.259854
Name: mean, dtype: float64


In [13]:
B['4Factor']

0.25985351962256253

In [14]:
team_stats['Season'].value_counts()

2019    10926
2018    10810
2017    10790
2016    10738
2014    10724
2015    10708
2020    10656
2013    10640
2010    10526
2012    10506
2009    10498
2011    10492
2008    10326
2007    10086
2006     9514
2005     9350
2003     9232
2004     9142
2021     7710
2022     7472
Name: Season, dtype: int64